In [ ]:
# #数据预览
# import pandas as pd

# df =pd.read_parquet('E:/GIT PROJECT/FR/kaggle/input/data/aeroclub-recsys-2025/train.parquet',engine='pyarrow').iloc[100:110]
# # print(df)


In [2]:
import dask.dataframe as dd

ddf1=dd.read_parquet('E:/GIT PROJECT/FR/kaggle/input/data/aeroclub-recsys-2025/processed_train.parquet')
#ddf2=dd.read_parquet('E:/GIT PROJECT/FR/kaggle/input/data/aeroclub-recsys-2025/test.parquet')
middle_rows1 = ddf1.partitions[0].compute().iloc[100:200]
#middle_rows2 = ddf2.partitions[0].compute().iloc[100:110]
middle_rows1.to_csv('middle_rows_dask3.csv',index=False)
#middle_rows2.to_csv('middle_rows_dask2.csv',index=False)

In [ ]:
import pandas as pd
import numpy as np 
from pathlib import Path
import pyarrow.parquet as pq
def load_data_in_chunks(file_path,chunksize):
    schema = pq.read_schema(file_path)
    dtype_mapping ={}
    for field in schema:
        col_name = field.name
        if field.type =='string':
            dtype_mapping[col_name]='category'
        elif 'int' in str(field.type):
            dtype_mapping[col_name]='Int32'  # Use pandas nullable integer type
        elif 'float' in str(field.type):
            dtype_mapping[col_name]='Float32'  # Use pandas nullable float type
    reader = pq.ParquetFile(file_path)
    for batch in reader.iter_batches(batch_size=chunksize):
        df_chunk = batch.to_pandas()
        for col, dtype in dtype_mapping.items():
            if col in df_chunk.columns:
                df_chunk[col] = df_chunk[col].astype(dtype)
        yield df_chunk


train = load_data_in_chunks('E:/GIT PROJECT/FR/kaggle/input/data/aeroclub-recsys-2025/train.parquet',chunksize=100000)
test =load_data_in_chunks('E:/GIT PROJECT/FR/kaggle/input/data/aeroclub-recsys-2025/test.parquet',chunksize=100000)

# Get the first chunk from each generator for preview
train_first_chunk = next(train)
test_first_chunk = next(test)

# print(f"训练集第一块形状: {train_first_chunk.shape}, 测试集第一块形状: {test_first_chunk.shape}")
# print("\n训练集列名:\n", train_first_chunk.columns.tolist())
# print("\n训练集前几行:\n", train_first_chunk.head())
print("\n缺失值统计:\n", train_first_chunk.isnull().sum().sort_values(ascending=False))


缺失值统计:
 legs0_segments3_marketingCarrier_code          100000
legs0_segments3_operatingCarrier_code          100000
legs0_segments3_aircraft_code                  100000
legs0_segments3_arrivalTo_airport_city_iata    100000
legs0_segments3_arrivalTo_airport_iata         100000
                                                ...  
searchRoute                                         0
sex                                                 0
taxes                                               0
totalPrice                                          0
selected                                            0
Length: 126, dtype: int64


In [6]:
import pandas as pd

# 设置Pandas显示选项，取消列名截断
pd.set_option('display.max_columns', None)  # 显示所有列
pd.set_option('display.max_rows', None)     # 显示所有行
pd.set_option('display.width', None)        # 自动调整宽度
pd.set_option('display.max_colwidth', None) # 显示完整的列内容

# 打印所有列名
print("所有因子名称:")
print(train_first_chunk.columns.tolist())  # 假设train是你的DataFrame

所有因子名称:
['Id', 'bySelf', 'companyID', 'corporateTariffCode', 'frequentFlyer', 'nationality', 'isAccess3D', 'isVip', 'legs0_arrivalAt', 'legs0_departureAt', 'legs0_duration', 'legs0_segments0_aircraft_code', 'legs0_segments0_arrivalTo_airport_city_iata', 'legs0_segments0_arrivalTo_airport_iata', 'legs0_segments0_baggageAllowance_quantity', 'legs0_segments0_baggageAllowance_weightMeasurementType', 'legs0_segments0_cabinClass', 'legs0_segments0_departureFrom_airport_iata', 'legs0_segments0_duration', 'legs0_segments0_flightNumber', 'legs0_segments0_marketingCarrier_code', 'legs0_segments0_operatingCarrier_code', 'legs0_segments0_seatsAvailable', 'legs0_segments1_aircraft_code', 'legs0_segments1_arrivalTo_airport_city_iata', 'legs0_segments1_arrivalTo_airport_iata', 'legs0_segments1_baggageAllowance_quantity', 'legs0_segments1_baggageAllowance_weightMeasurementType', 'legs0_segments1_cabinClass', 'legs0_segments1_departureFrom_airport_iata', 'legs0_segments1_duration', 'legs0_segments1_fli

In [4]:
import pyarrow.parquet as pq
import pandas as pd

def analyze_null_columns(parquet_path, threshold=99):
    """
    分析Parquet文件中空值比例过高的特征
    
    参数:
        parquet_path: Parquet文件路径
        threshold: 空值比例阈值(百分比)，默认99%
    
    返回:
        DataFrame包含空值比例超过阈值的特征及其统计信息
    """
    # 1. 打开Parquet文件
    parquet_file = pq.ParquetFile(parquet_path)
    
    # 2. 获取列名和初始化计数器
    schema = parquet_file.schema_arrow  # 使用schema_arrow替代schema
    columns = schema.names
    null_counts = {col: 0 for col in columns}
    total_rows = parquet_file.metadata.num_rows
    
    print(f"开始分析文件: {parquet_path}")
    print(f"总行数: {total_rows:,}")
    print(f"总特征数: {len(columns)}")
    
    # 3. 分批次读取
    batch_size = 5000  # 根据内存调整
    processed_rows = 0
    
    for batch in parquet_file.iter_batches(batch_size=batch_size):
        df_batch = batch.to_pandas()
        processed_rows += len(df_batch)
        
        # 更新进度信息
        progress = (processed_rows / total_rows) * 100
        print(f"\r处理进度: {progress:.1f}% ({processed_rows:,}/{total_rows:,}行)", end="", flush=True)
        
        for col in columns:
            if col in df_batch.columns:
                null_counts[col] += df_batch[col].isnull().sum()
    
    print("\n分析完成!")
    
    # 4. 计算空值比例
    null_percent = {col: (null_counts[col] / total_rows) * 100 for col in columns}
    
    # 5. 创建结果DataFrame
    result_df = pd.DataFrame({
        'feature': columns,
        'null_count': [null_counts[col] for col in columns],
        'null_percent': [null_percent[col] for col in columns],
        'dtype': [str(schema.field(col).type) for col in columns]  # 使用schema.field
    })
    
    # 6. 筛选并排序高比例空值特征
    high_null_df = result_df[result_df['null_percent'] > threshold]\
                     .sort_values('null_percent', ascending=False)
    
    return high_null_df

# 使用示例
try:
    high_null_features = analyze_null_columns(
        'E:/GIT PROJECT/FR/kaggle/input/data/aeroclub-recsys-2025/processed_train.parquet',
        threshold=-1  # 可以调整此阈值
    )

    print(len(high_null_features))
    # 打印结果
    if not high_null_features.empty:
        print("\n空值比例超过阈值的特征:")
        print(high_null_features[['feature', 'null_percent', 'dtype']].to_string(index=False))
        
        # 保存结果到CSV
        high_null_features.to_csv('high_null_features1.csv', index=False)
        print("\n结果已保存到 high_null_features.csv")
    else:
        print("\n没有发现空值比例超过阈值的特征")
        
except Exception as e:
    print(f"发生错误: {str(e)}")

开始分析文件: E:/GIT PROJECT/FR/kaggle/input/data/aeroclub-recsys-2025/processed_train.parquet
总行数: 18,145,372
总特征数: 102
处理进度: 100.0% (18,145,372/18,145,372行)
分析完成!
102

空值比例超过阈值的特征:
                                               feature  null_percent  dtype
                                                    Id           0.0  int32
                                                bySelf           0.0   bool
                                             companyID           0.0  int32
                                   corporateTariffCode           0.0  int16
                                         frequentFlyer           0.0  int32
                                           nationality           0.0   int8
                                            isAccess3D           0.0   bool
                                                 isVip           0.0   bool
                                       legs0_arrivalAt           0.0  int32
                                     legs0_departureAt         

初始内存占用: 209.91 MB
⏳ 加载数据...
数据加载后内存占用: 19814.77 MB
⚠️ 内存过高，自动采样前1000000行

🔧 执行特征工程...


特征工程进度:   0%|          | 0/100 [00:00<?, ?it/s]

初始内存占用: 944.14 MB


特征工程进度:  20%|██        | 20/100 [00:03<00:12,  6.36it/s]

内存优化完成，减少 71.3% (最终: 270.82 MB)


特征工程进度:  45%|████▌     | 45/100 [00:03<00:04, 12.37it/s]


❌ 处理失败: 'Series' object has no attribute 'ast'
调试建议:
1. 检查数据路径是否正确
2. 使用train.head(1000).info()检查数据结构
3. 检查Python环境版本（建议3.8+）


E:/GIT PROJECT/FR/kaggle/input/data/aeroclub-recsys-2025/train.parquet